In [27]:
from caremel_predict.interfaces.data_interactor import DataInteractor
import collections
import numpy as np
import pandas as pd

In [53]:
di = DataInteractor()

def _add_in_dict_ds(row):
    try:
        ds_dict[row['member_internal_code']] = {
            **ds_dict[row['member_internal_code']],
            **{
                row['cid']: 1
            }
        }
    except:
        ds_dict[row['member_internal_code']] = {row['cid']: 1}

def _add_in_dict_fp(row):
    if row['question_text'] in QUESTIONS:
        try:
            fp_dict[row['member_internal_code']] = {
                **fp_dict[row['member_internal_code']],
                **{
                    row['question_text']: row['answer_text']
                }
            }
        except:
            fp_dict[row['member_internal_code']] = {
                row['question_text']: row['answer_text']
            }


In [38]:
onboarding_df = di.redshift.run_str_query(
    query="""
WITH 

motivo_consulta AS (
        WITH
        dados_iniciais AS (
        SELECT
            member_internal_code,
            active_membership_since,
            membership_phi_sk,
            updated_at AS onboarding_updated_at,
            status AS status_onboarding
        FROM
            curated.alice_memberships_phi amp
        FULL OUTER JOIN
            ods.member_onboarding_checkpoint_current mocc on amp.personid_03 = mocc.person_id 
            AND (((amp.active_membership_since <= dateadd(d,1,mocc.updated_at::timestamp) and membership_status = 'ACTIVE') OR updated_at is null ) 
                OR ((amp.active_membership_since <= dateadd(d,1,mocc.updated_at::timestamp) and dateadd(d,1,amp.canceled_at::timestamp) >= mocc.updated_at::date and membership_status = 'CANCELED') OR mocc.updated_at is null))
        WHERE
            (acquisition_type in ('New Member','Recovered Member') or acquisition_type is null)
            AND (active_membership_since>'2022-09-18' or (active_membership_since is null and updated_at is not null))
        ),

        -- Filtrar o último dado do membro antes de data de finalizar o onboarding
        risco_onboarding as (
        SELECT DISTINCT
            member_internal_code,
            onboarding_updated_at,
            risk_description AS risco_onboarding,
            final_value AS valor_risco_onboarding,
            added_at AS data_risco_onboarding,
            'Valor risco' as fator,
            CASE WHEN final_value>0 then final_value::text 
                ELSE null 
            END AS valor,
            null as condicao_associada
        FROM 
            (SELECT
                *,
                row_number() over (partition by rpc.member_internal_code order by added_at desc) as rn
            FROM
                risk_public_current rpc
            LEFT JOIN
                dados_iniciais di USING (member_internal_code)
            WHERE 
                (rpc.added_at::date <= dateadd(d,1,onboarding_updated_at::date) AND onboarding_updated_at IS NOT NULL) OR rpc.added_at IS NULL)
        WHERE
            rn = 1),

        -- Filtrar o último dado do membro antes de data de finalizar o onboarding
        cids_onboarding AS (
        SELECT DISTINCT
            person_internal_code as member_internal_code,
            active_membership_since,
            onboarding_updated_at,
            disease_code_value as code,
            --age,
            severity
        FROM
            (SELECT
                *,
                row_number() over (partition by person_internal_code, disease_code_value order by added_at desc) as rn
            FROM 
                case_record_current crc
            LEFT JOIN
                dados_iniciais di on di.member_internal_code = crc.person_internal_code 
            WHERE
                disease_code_type = 'CID_10'
                AND ((dateadd(d,1,di.onboarding_updated_at::date) >= crc.added_at and status_onboarding = 'ENDED') OR status_onboarding <> 'ENDED' OR status_onboarding IS NULL)
            ) aux
        WHERE
            rn =1 AND status <> 'CANCELLED' AND severity <> 'INACTIVE'),
            
        cid_risco AS (
        SELECT DISTINCT
            member_internal_code,
            onboarding_updated_at,
            'CID risco' as fator,
            listagg(code,'; ') AS valor,
            listagg(concat('CID: ',code),'; ') AS condicao_associada
        FROM
            cids_onboarding
        WHERE 
            code IN ('E05','E062','E063','E064','E065','E079','E03','E030','E031','E032','E035','E038','E039','F45','F451','F711','G40','G400','G401','G402','G403','G404','G405','G83','G830','G831','G832','G833','G834','G838','G839','I425','I428','I438','I460','I469','I49','I498','I499','I52','I67','I688','I69','I690','I691','I692','I693','I698','I72','I722','I728','I770','I790','I85','I864','I971','J410','J42','J43','J431','J432','J438','J439','K221','K222','K224','K225','K227','K228','K231','K25','K254','K255','K256','K257','K259','K26','K51','K510','K511','K512','K513','K518','K519','K700','K702','K731','K732','K738','K746','K760','K900','K904','K91','K911','K912','K921','M623','C46','C460','C461','C462','C463','C467','C468','C469','D50','D500','D501','D508','D509','D51','D510','D511','D512','D513','D518','D519','D52','D520','D521','D528','D529','D53','D530','D531','D532','D538','D539','E40','E41','E42','E43','E44','E440','E441','E45','E46','E78','E780','E781','E782','E783','E784','E785','E786','E788','E789')
        GROUP BY
            member_internal_code,onboarding_updated_at),
            
        -- Filtrar o primeiro dado do membro depois da respectiva membership
        forms_filter_membership as (
        SELECT DISTINCT
            member_internal_code,
            active_membership_since,
            form_name,
            index,
            summary_question,
            answer_text,
            answer_time,
            CASE WHEN form_name = 'Questionário Onboarding 1.0' AND index = 22 AND answer_text LIKE '%quando tinha < 50 anos de idade%' THEN 'Hist_Cancer_Intestino'
                WHEN form_name = 'Questionário Onboarding 1.0' AND index = 21 AND answer_text LIKE '%quando tinha < 50 anos de idade%' THEN 'Hist_Cancer_Mama'
                WHEN form_name = 'Questionário Onboarding 1.0' AND index in (86,85,84,83) AND answer_text = 'Sim' THEN 'PAR-Q'
                WHEN form_name = 'Questionário Onboarding 1.0' AND index = 13 AND answer_text='Sim' THEN 'Gestante'
                WHEN form_name = 'Questionário Saúde Mental' AND index = 16 AND answer_text::int>0 THEN 'Questão 9 PHQ-9'
            END AS fator,
            CASE WHEN form_name = 'Questionário Onboarding 1.0' AND index in (22,21) AND answer_text LIKE '%quando tinha < 50 anos de idade%' THEN answer_text
                WHEN form_name = 'Questionário Onboarding 1.0' AND index in (86,85,84,83) AND answer_text = 'Sim' THEN answer_text
                        WHEN form_name = 'Questionário Onboarding 1.0' AND index = 13 AND answer_text='Sim' THEN answer_text
                WHEN form_name = 'Questionário Saúde Mental' AND index = 16 AND answer_text::int>0 THEN answer_text
            END AS valor,
            CASE WHEN form_name = 'Questionário Onboarding 1.0' AND index in (22,21) AND answer_text LIKE '%quando tinha < 50 anos de idade%' THEN 'CIAP A21'
                WHEN form_name = 'Questionário Onboarding 1.0' AND index in (86,85,84,83) AND answer_text = 'Sim' THEN 'CIAP A23'
                WHEN form_name = 'Questionário Onboarding 1.0' AND index = 13 AND answer_text='Sim' THEN 'CIAP W78'
                WHEN form_name = 'Questionário Saúde Mental' AND index = 16 AND answer_text::int>0 THEN 'CIAP P03'
            END AS condicao_associada
        FROM
            (SELECT
                *,
                row_number() OVER (PARTITION BY member_internal_code, form_name, index ORDER BY answer_time) as rn
            FROM
                health_form_current
            LEFT JOIN
                dados_iniciais USING (member_internal_code)
            WHERE
                dateadd(d,1,answer_time::date)>=active_membership_since
                AND ((form_name = 'Questionário Onboarding 1.0' and index in (13,21,22,83,84,85,86)) or form_name = 'Questionário Saúde Mental' and index=16)
            )
        WHERE
            rn = 1),
            
        -- Filtrar o primeiro dado do membro depois da respectiva membership
        outcome_filter_membership as (
        SELECT DISTINCT
            member_internal_code,
            active_membership_since,
            outcome_type,
            outcome_value,
            added_at,
            CASE WHEN outcome_type = 'IMC' AND ((outcome_value<=18.5 AND outcome_value>10) OR (outcome_value>=30)) then 'IMC'
                WHEN outcome_type = 'GAD_7' AND outcome_value>9 then 'GAD 7'
                WHEN outcome_type = 'PHQ_9' AND outcome_value>9 then 'PHQ 9'
            END AS fator,
            CASE WHEN outcome_type = 'IMC' AND ((outcome_value<=18.5 AND outcome_value>10) OR (outcome_value>=30)) then round(outcome_value,1)::text
                WHEN outcome_type in ('GAD_7','PHQ_9') AND outcome_value>9 then outcome_value::int::text
            END AS valor,
            CASE WHEN outcome_type = 'IMC' AND outcome_value<=18.5 AND outcome_value>10 then 'CIAP T91'
                WHEN outcome_type = 'IMC' AND (outcome_value>=30) then 'CIAP T82'
                WHEN outcome_type = 'GAD_7' AND outcome_value>9 then 'CIAP P01'
                WHEN outcome_type = 'PHQ_9' AND outcome_value>9 then 'CIAP P03'
            END AS condicao_associada
        FROM
            (SELECT
                *,
                row_number() OVER (PARTITION BY member_internal_code, outcome_type ORDER BY added_at) as rn
            FROM
                clinical_outcome_record
            LEFT JOIN
                dados_iniciais USING (member_internal_code)
            WHERE
                dateadd(d,1,added_at::date)>=active_membership_since 
                AND outcome_type in ('PHQ_9','GAD_7','IMC')
            )
        WHERE
            rn = 1),


        consolidado_motivos AS (
        SELECT
            member_internal_code,
            active_membership_since,
            onboarding_updated_at,
            fator,
            valor,
            condicao_associada
        FROM
            dados_iniciais
        INNER JOIN
            risco_onboarding USING (member_internal_code,onboarding_updated_at)
        WHERE
            valor IS NOT NULL
        UNION ALL
        SELECT
            member_internal_code,
            active_membership_since,
            onboarding_updated_at,
            fator,
            valor,
            condicao_associada
        FROM
            dados_iniciais
        INNER JOIN
            cid_risco USING (member_internal_code,onboarding_updated_at)
        UNION ALL
        SELECT
            member_internal_code,
            active_membership_since,
            onboarding_updated_at,
            fator,
            valor,
            condicao_associada
        FROM
            dados_iniciais
        INNER JOIN
            forms_filter_membership USING (member_internal_code,active_membership_since)
        WHERE
            fator IS NOT NULL
        UNION ALL
        SELECT
            member_internal_code,
            active_membership_since,
            onboarding_updated_at,
            fator,
            valor,
            condicao_associada
        FROM
            dados_iniciais
        INNER JOIN
            outcome_filter_membership USING (member_internal_code,active_membership_since)
        WHERE
            fator IS NOT NULL)
            
        SELECT DISTINCT
            member_internal_code,
            active_membership_since,
            membership_phi_sk,
            onboarding_updated_at as status_onboarding_updated_at,
            risco_onboarding as onboarding_risk,
            valor_risco_onboarding as onboarding_risk_value,
            data_risco_onboarding  as onboarding_risk_added_at,
            consolidado_motivos.fator as type,
            consolidado_motivos.valor as value,
            consolidado_motivos.condicao_associada as demand,
            total_type_count
        /*    CASE WHEN (risco_onboarding in ('MEDIUM_RISK','HIGH_RISK')) and consolidado_motivos.fator <> 'Gestante' then Null
                ELSE consolidado_motivos.fator end as type,
            CASE WHEN (risco_onboarding in ('MEDIUM_RISK','HIGH_RISK')) and consolidado_motivos.fator <> 'Gestante' then Null
                ELSE consolidado_motivos.valor end as value,
            CASE WHEN (risco_onboarding in ('MEDIUM_RISK','HIGH_RISK')) and consolidado_motivos.fator <> 'Gestante' then Null
                ELSE consolidado_motivos.condicao_associada end as demand*/
        FROM 
            dados_iniciais
        LEFT JOIN
            risco_onboarding USING (member_internal_code,onboarding_updated_at)
        LEFT JOIN
            consolidado_motivos USING (member_internal_code,active_membership_since,onboarding_updated_at)
        LEFT JOIN
            (SELECT member_internal_code,count(*) as total_type_count FROM consolidado_motivos GROUP BY member_internal_code) freq_fatores USING (member_internal_code)
        WHERE member_internal_code IS NOT NULL and (status_onboarding = 'ENDED' or status_onboarding is null)
    ),




finalizou_quest AS (
        SELECT
            hfc.member_internal_code,
            active_membership_since,
            membership_phi_sk,
            min(answer_time) AS onboarding_form_ended_at
        FROM
            health_form_current hfc
        LEFT JOIN
            curated.alice_memberships_phi amp on amp.member_internal_code = hfc.member_internal_code 
            AND ((dateadd(d,1,amp.canceled_at::timestamp) >= answer_time and membership_status = 'CANCELED') or membership_status = 'ACTIVE')
        WHERE
            form_name = 'Questionário Onboarding 1.0' AND index = 98
        GROUP BY 
            hfc.member_internal_code,active_membership_since,membership_phi_sk
),

outcome_sm_membership as (
        SELECT DISTINCT
            member_internal_code,
            active_membership_since,
            membership_phi_sk,
            1 AS flag_score_mental
        FROM
                (
                SELECT
                    *,
                    row_number() OVER (PARTITION BY member_internal_code, outcome_type ORDER BY added_at) as rn
                FROM
                    score_magenta_outcomes
                LEFT JOIN
                    motivo_consulta dados USING (member_internal_code)
                WHERE
                    dateadd(d,1,added_at::timestamp)>=active_membership_since 
                    AND outcome_value<750 and outcome_type='MENTAL_SCORE_MAGENTA'
                )
        WHERE
            rn = 1
),

motivo_concat AS (
        SELECT DISTINCT
            member_internal_code,
            active_membership_since,
            membership_phi_sk,
            status_onboarding_updated_at,
            onboarding_risk,
            onboarding_risk_value,
            onboarding_risk_added_at,
            listagg(type||': '||value,'; ') AS appointment_reasons,
            listagg(demand,'; ') AS appointment_demands
        FROM
            motivo_consulta AS motivo
        GROUP BY
            1,2,3,4,5,6,7
),

geral AS (
    SELECT
        member_internal_code,
        datediff(year,date_of_birth::Date,current_timestamp::Date) as age,
        personid_03 as person_id,
        membership_phi_sk,
        active_membership_since,
        membership_status,
        membership_count,
        total_membership_count,
        canceled_at,
        acquisition_type,
        updated_at AS status_onboarding_updated_at,
        status AS status_onboarding,
        onboarding_form_ended_at,
        onboarding_risk,
        onboarding_risk_value,
        onboarding_risk_added_at,
        appointment_reasons,
        appointment_demands,
        flag_score_mental
    FROM
        curated.alice_memberships_phi amp
    FULL OUTER JOIN
        ods.member_onboarding_checkpoint_current mocc 
        ON amp.personid_03 = mocc.person_id 
        AND (((amp.active_membership_since <= dateadd(h,4,mocc.updated_at::timestamp) and membership_status = 'ACTIVE') OR updated_at is null) 
                OR ((amp.active_membership_since <= dateadd(h,4,mocc.updated_at::timestamp) and dateadd(h,4,amp.canceled_at::timestamp) >= mocc.updated_at::date and membership_status = 'CANCELED') OR mocc.updated_at is null))
    FULL OUTER JOIN
        finalizou_quest using (member_internal_code,active_membership_since,membership_phi_sk)
    LEFT JOIN
        motivo_concat using (member_internal_code,active_membership_since,membership_phi_sk)
    LEFT JOIN
        outcome_sm_membership using (member_internal_code,active_membership_since,membership_phi_sk)
    WHERE
        (acquisition_type in ('New Member','Recovered Member') or acquisition_type is null)
        AND (active_membership_since>'2022-09-18' or (active_membership_since is null and status_onboarding_updated_at is not null))
    ORDER BY
        active_membership_since
),

geral_v2 AS(
    SELECT
    member_internal_code,
    age,
    person_id,
    membership_phi_sk,
    active_membership_since,
    membership_status,
    membership_count,
    total_membership_count,
    canceled_at,
    acquisition_type,
    status_onboarding_updated_at,
    status_onboarding,
    onboarding_form_ended_at,
    case 
        when age<12 then 'Criança'
        when onboarding_form_ended_at is null then 'Não respondeu onboarding'
        else 'Respondeu'
    end as status_onboarding_form,
    case 
        when age<12 then 'Criança'
        when (onboarding_form_ended_at is null and status_onboarding is null) or (onboarding_form_ended_at is null and status_onboarding<>'ENDED') then 'Não Finalizou'
        when onboarding_form_ended_at is not null AND status_onboarding='ENDED' then 'Finalizou'
        else 'Indefinido'
    end as status_onboarding_curated,
    onboarding_risk,
    onboarding_risk_value,
    onboarding_risk_added_at,
    CASE 
        WHEN age<12 then 'Criança'
        WHEN onboarding_risk in ('MEDIUM_RISK','HIGH_RISK') THEN 'Imersão'
        WHEN appointment_reasons like '%Gestante%' then 'Gestante'
        WHEN age>11 and age<18 then 'Adolescente (12-18a)'
        WHEN appointment_reasons is not null THEN 'Consulta 20 min'
        WHEN onboarding_risk='NO_RISK' or onboarding_risk is null THEN 'Jornada Bem-Estar'
    END AS follow_up_expected,
    CASE 
        WHEN age<12 then 'Sem tarefa'
        WHEN onboarding_risk in ('MEDIUM_RISK','HIGH_RISK') THEN 'Sem tarefa'
        WHEN appointment_reasons like '%Gestante%' then 'Enf Atendimento - Consulta Digital Gestante'
        WHEN age>11 and age<18 then 'Enf Referência'
        WHEN appointment_reasons is not null THEN 'Med - Consulta Longitudinal'
        WHEN onboarding_risk='NO_RISK' or onboarding_risk is null THEN 'PDA Pilar'
    END AS health_plan_task_expected,
    CASE 
        WHEN flag_score_mental= 1 and appointment_reasons is not null and (appointment_reasons not like '%GAD%' or appointment_reasons not like '%PHQ%') then appointment_reasons ||'; Aplicar/ verificar se membro respondeu o GAD 7/PHQ 9 (olhar no histórico clínico > questionários)'
        ELSE appointment_reasons
    END AS appointment_orientations,
    appointment_demands,
    flag_score_mental

    FROM
    geral
    WHERE
    active_membership_since>'2022-09-18' and not (membership_status='CANCELED' AND datediff(d,active_membership_since,canceled_at)<7)
)

SELECT * FROM geral_v2
"""
)

/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [192]:
df_fp = di.redshift.run_str_query(
    query="""
WITH 
questionarios AS(
    SELECT
    hfc.member_internal_code,
    hfc.member_sex,
    hfc.member_year_of_birth,
    hfc.question_text,
    hfc.answer_text
    
    FROM
        health_form_current hfc
    WHERE
            form_name = 'Questionário Onboarding 1.0' 
        AND (
                hfc.question_text= 'Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text = 'Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'
                OR hfc.question_text ='O que te incomoda em relação à sua saúde?'
                OR hfc.question_text ='Qual a sua altura? (em cm)'
                OR hfc.question_text ='Qual o seu peso atual? (em kg)'
                OR hfc.question_text ='Você fuma ou já fumou?'
                OR hfc.question_text ='Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?'    
                
            )
    )
    
SELECT * FROM questionarios
"""
)
df_fp.head(10)

/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,member_internal_code,member_sex,member_year_of_birth,question_text,answer_text
0,NC1J5EE,FEMALE,1995,Alguma dessas doenças está presente na sua fam...,Não
1,NC149F7,FEMALE,1991,Alguma dessas doenças está presente na sua fam...,Não
2,NC1JIOX,FEMALE,1990,Alguma dessas doenças está presente na sua fam...,Não
3,NC11C1K,FEMALE,1995,Alguma dessas doenças está presente na sua fam...,Não
4,NC147TG,FEMALE,1991,Alguma dessas doenças está presente na sua fam...,Não
5,NC1CELR,FEMALE,1999,Você usa drogas recreativas (como maconha ou o...,Não
6,NC1JIOX,FEMALE,1990,Alguma dessas doenças está presente na sua fam...,Não
7,NC1OTLA,FEMALE,1997,Alguma dessas doenças está presente na sua fam...,Não
8,NC19VN2,MALE,1984,Alguma dessas doenças está presente na sua fam...,Não
9,NC1G5GY,MALE,1991,Alguma dessas doenças está presente na sua fam...,null


In [193]:
# local
#di.static
#query on redshift
df_ds = di.redshift.run_str_query(query="""
    WITH dados_iniciais AS (SELECT
    member_internal_code,
    active_membership_since,
    membership_phi_sk,
    updated_at AS onboarding_updated_at,
    status AS status_onboarding
FROM
    curated.alice_memberships_phi amp
FULL OUTER JOIN
    ods.member_onboarding_checkpoint_current mocc on amp.personid_03 = mocc.person_id 
    AND (((amp.active_membership_since <= dateadd(d,1,mocc.updated_at::timestamp) and membership_status = 'ACTIVE') OR updated_at is null ) 
        OR ((amp.active_membership_since <= dateadd(d,1,mocc.updated_at::timestamp) and dateadd(d,1,amp.canceled_at::timestamp) >= mocc.updated_at::date and membership_status = 'CANCELED') OR mocc.updated_at is null))
WHERE
    (acquisition_type in ('New Member','Recovered Member') or acquisition_type is null)
    AND (active_membership_since>'2022-09-18' or (active_membership_since is null and updated_at is not null))
),

-- Filtrar o último dado do membro antes de data de finalizar o onboarding
risco_onboarding as (
SELECT DISTINCT
    member_internal_code,
    onboarding_updated_at,
    risk_description AS risco_onboarding,
    final_value AS valor_risco_onboarding,
    added_at AS data_risco_onboarding,
    'Valor risco' as fator,
    CASE WHEN final_value>0 then final_value::text 
         ELSE null 
    END AS valor,
    null as condicao_associada
FROM 
    (SELECT
        *,
        row_number() over (partition by rpc.member_internal_code order by added_at desc) as rn
    FROM
        risk_public_current rpc
    LEFT JOIN
        dados_iniciais di USING (member_internal_code)
    WHERE 
        (rpc.added_at::date <= dateadd(d,1,onboarding_updated_at::date) AND onboarding_updated_at IS NOT NULL) OR rpc.added_at IS NULL)
WHERE
    rn = 1),

-- Filtrar o último dado do membro antes de data de finalizar o onboarding
cids_onboarding AS (
SELECT DISTINCT
    person_internal_code as member_internal_code,
    active_membership_since,
    onboarding_updated_at,
    disease_code_value as code,
    --age,
    severity
FROM
    (SELECT
        *,
        row_number() over (partition by person_internal_code, disease_code_value order by added_at desc) as rn
    FROM 
        case_record_current crc
    LEFT JOIN
        dados_iniciais di on di.member_internal_code = crc.person_internal_code 
    WHERE
        disease_code_type = 'CID_10'
        AND ((dateadd(d,1,di.onboarding_updated_at::date) >= crc.added_at and status_onboarding = 'ENDED') OR status_onboarding <> 'ENDED' OR status_onboarding IS NULL)
    ) aux
WHERE
    rn =1 AND status <> 'CANCELLED' AND severity <> 'INACTIVE')
    
SELECT DISTINCT 
member_internal_code,
code as cid
 from cids_onboarding
"""
)
#run_sql is to ran a sql file
#


/Users/luiz.superti/Library/Python/3.9/lib/python/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [31]:
df_ds['cid'] = df_ds['cid'].apply(lambda x: x[0:3])
ds_dict = collections.defaultdict()

df_ds.apply(lambda row: _add_in_dict_ds(row), axis=1)
df_ds_row = pd.DataFrame.from_dict(ds_dict, orient='index').reset_index(drop=False).rename(columns={'index': 'member_internal_code'})
df_ds_row = df_ds_row.merge(df_ds)

df_ds_row.fillna(0, inplace=True)
df_ds_row.drop(columns='cid', inplace=True)
df_ds_row['is_risk_or_chronic_dis'] = np.where(
(df_ds_row['E66'] == 1) | (df_ds_row['N63'] == 1) |
(df_ds_row['I10'] == 1) | (df_ds_row['E05'] == 1) |
(df_ds_row['E11'] == 1) | (df_ds_row['B24'] == 1) |
(df_ds_row['F84'] == 1) | (df_ds_row['M05'] == 1) |
(df_ds_row['Q90'] == 1) | (df_ds_row['I64'] == 1) |
(df_ds_row['I50'] == 1) | (df_ds_row['N18'] == 1) |
(df_ds_row['Z95'] == 1), 1, 0)

df_ds_row.drop_duplicates(inplace=True)


In [194]:
fp_dict = collections.defaultdict()
df_fp.apply(lambda row: _add_in_dict_fp(row), axis=1)
df_fp_row = pd.DataFrame.from_dict(fp_dict, orient='index').reset_index(drop=False).rename(
                                                             columns={'index': 'member_internal_code'})
df_fp_row.fillna(-1, inplace=True)

In [195]:
df_fp_row['has_ancestry_blood_pressure'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_blood_pressure'] = 0

######################################################
df_fp_row['has_ancestry_heart_attack0'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Infarto do miocárdio -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_heart_attack0'] = 0

df_fp_row['has_ancestry_heart_attack1'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_heart_attack1'] = 0


####cancers
df_fp_row['has_ancestry_cancer0'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer0'] = 0


df_fp_row['has_ancestry_cancer1'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer1'] = 0

df_fp_row['has_ancestry_cancer2'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.']=='Não','has_ancestry_cancer2'] = 0



df_fp_row['has_ancestry_diabetes'] = np.where((df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Avós"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Pai"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Mãe"))|(
            df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.'].str.contains("Irmãos")),1,-1)
df_fp_row.loc[df_fp_row['Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.']=='Não', 'has_ancestry_diabetes'] = 0


In [196]:
df_fp_row['has_ancestry_heart_attack'] = np.where((df_fp_row['has_ancestry_heart_attack0']==1)|(df_fp_row['has_ancestry_heart_attack1']==1),1,-1)
df_fp_row.loc[(df_fp_row['has_ancestry_heart_attack0']==0)&(df_fp_row['has_ancestry_heart_attack1']==0),'has_ancestry_heart_attack'] = 0

df_fp_row['has_ancestry_cancer'] = np.where((df_fp_row['has_ancestry_cancer0']==1)|(df_fp_row['has_ancestry_cancer1']==1)|(df_fp_row['has_ancestry_cancer2']==1),1,-1)
df_fp_row.loc[(df_fp_row['has_ancestry_cancer0']==0)&(df_fp_row['has_ancestry_cancer1']==0)&(df_fp_row['has_ancestry_cancer2']==0),'has_ancestry_cancer'] = 0


In [197]:
df_fp_row['has_incomodo_para_tratar_len'] = df_fp_row['O que te incomoda em relação à sua saúde?'].apply(lambda x: len(x))
df_fp_row['v1'] = df_fp_row['O que te incomoda em relação à sua saúde?'].apply(lambda x:str.lower(x))
df_fp_row['has_incomodo_para_tratar'] = np.where((df_fp_row['has_incomodo_para_tratar_len']>3) & (
                                                    df_fp_row['v1']!='nada') &  (
                                                    df_fp_row['v1']!='não sei') & (
                                                    df_fp_row['v1']!='nada em específico') & (
                                                    df_fp_row['v1']!='por enquanto nada') & (
                                                    df_fp_row['v1']!='no momento nada') & (
                                                    df_fp_row['v1']!='nada em particular') & (
                                                    df_fp_row['v1']!='nada.') & (
                                                    df_fp_row['v1']!='ainda nada') & (
                                                    df_fp_row['v1']!='por enquanto nada.') & (
                                                    df_fp_row['v1']!='por enquanto, nada.') & (
                                                    df_fp_row['v1']!='nada até o momento') & (
                                                    df_fp_row['v1']!='Ainda nada') & (
                                                    df_fp_row['v1']!='No momento nada :)'
                                                    ),1,0)
df_fp_row.loc[df_fp_row['O que te incomoda em relação à sua saúde?'] == 'null', 'has_incomodo_para_tratar'] = -1

In [198]:
df_fp_row['Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?'].value_counts()

Não                                                                                                                                                 616
Sim                                                                                                                                                 107
-1                                                                                                                                                   40
Sim@Maconha                                                                                                                                          12
null                                                                                                                                                  5
Sim@Apenas maconha, ocasionalmente (+- uma vez por mês). Não gosto de usar drogas sintéticas.                                                         1
Sim@Maconha raramente                                                                   

In [204]:
# Fumo
df_fp_row['smoke'] = np.where(df_fp_row['Você fuma ou já fumou?']=='10',0,1)
df_fp_row.loc[df_fp_row['Você fuma ou já fumou?'] == '-1', ['smoke']] = -1

# drogas
df_fp_row['v1'] = df_fp_row['Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?'].apply(lambda x: str(x).partition('@')[0])
df_fp_row['drug'] = np.where(df_fp_row['v1']=='Não',0,1)
df_fp_row.loc[df_fp_row['v1'] == "-1", ['drug']] = -1
df_fp_row.loc[df_fp_row['v1'] == "null", ['drug']] = -1


In [211]:
df_fp_row['imc'] = df_fp_row['Qual o seu peso atual? (em kg)'].apply(lambda x: float(x)) / (df_fp_row['Qual a sua altura? (em cm)'].apply(lambda x:float(x))/100)**2
df_fp_row['imc'] = df_fp_row['imc'].apply(lambda x: round(x,2))
df_fp_row['imc'] = df_fp_row['imc'].fillna(-1)

In [207]:
df_fp_row.columns

Index(['member_internal_code',
       'Alguma dessas doenças está presente na sua família: Hipertensão arterial -  Pode selecionar todas as opções aplicáveis.',
       'Alguma dessas doenças está presente na sua família: Câncer de pele -  Pode selecionar todas as opções aplicáveis.',
       'Você usa drogas recreativas (como maconha ou outras drogas sintéticas)?',
       'Qual a sua altura? (em cm)',
       'Alguma dessas doenças está presente na sua família: Câncer de Mama -  Pode selecionar todas as opções aplicáveis.',
       'Qual o seu peso atual? (em kg)', 'Você fuma ou já fumou?',
       'Alguma dessas doenças está presente na sua família: Acidente vascular cerebral (derrame) -  Pode selecionar todas as opções aplicáveis.',
       'Alguma dessas doenças está presente na sua família: Câncer de Intestino -  Pode selecionar todas as opções aplicáveis.',
       'Alguma dessas doenças está presente na sua família: Diabetes -  Pode selecionar todas as opções aplicáveis.',
       'O qu